In [2]:
import json
import uproot
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema, BaseSchema
from coffea import nanoevents
from coffea import processor
import time

import argparse
import warnings
import pyarrow as pa
import pyarrow.parquet as pq
import pickle as pkl
import pandas as pd
import os
import sys
sys.path.append("../")

from collections import defaultdict
import pickle as pkl
import pyarrow as pa
import awkward as ak
import numpy as np
import pandas as pd
import json
import os
import shutil
import pathlib
from typing import List, Optional
import pyarrow.parquet as pq

from coffea import processor
from coffea.nanoevents.methods import candidate, vector
from coffea.analysis_tools import Weights, PackedSelection

import warnings
warnings.filterwarnings("ignore", message="Found duplicate branch ")
warnings.filterwarnings("ignore", category=DeprecationWarning)
np.seterr(invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [3]:
! ls /eos/uscms/store/user/lpcpfnano/cmantill/v2_4/2017/*

/eos/uscms/store/user/lpcpfnano/cmantill/v2_4/2017/HWW:
GluGluHToWW_Pt-200ToInf_M-125_TuneCP5_MINLO_13TeV-powheg-pythia8
GluGluToHHTobbVV_node_cHHH1_TuneCP5_13TeV-powheg-pythia8

/eos/uscms/store/user/lpcpfnano/cmantill/v2_4/2017/QCD:
QCD_HT1000to1500_TuneCP5_PSWeights_13TeV-madgraph-pythia8
QCD_Pt_1800to2400_TuneCP5_13TeV_pythia8

/eos/uscms/store/user/lpcpfnano/cmantill/v2_4/2017/TTbar:
TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8


In [11]:
# load a root file into coffea-friendly NanoAOD structure
import uproot
fn = "/eos/uscms/store/user/lpcpfnano/cmantill/v2_4/2017/HWW/GluGluHToWW_Pt-200ToInf_M-125_TuneCP5_MINLO_13TeV-powheg-pythia8/GluGluHToWW_Pt-200ToInf_M-125/230203_210825/0000/nano_mc2017_1-1.root"
f = uproot.open(fn)
num = f['Events'].num_entries   ### checks number of events per file 
print(f'number of events per file is {num}')

events = nanoevents.NanoEventsFactory.from_root(f, "Events").events()

number of events per file is 6000


In [27]:
for key in f["Events"].keys():
    if "incl" in key:
        print(key)

FatJet_inclParTMDV1_HWW4q3qvsQCD
FatJet_inclParTMDV1_HWW4qvsQCD
FatJet_inclParTMDV1_HWWlvqqvsQCDTop
FatJet_inclParTMDV1_HWWlvqqvsTopblv
FatJet_inclParTMDV1_HbbvsQCD
FatJet_inclParTMDV1_HccvsQCD
FatJet_inclParTMDV1_hidNeuron000
FatJet_inclParTMDV1_hidNeuron001
FatJet_inclParTMDV1_hidNeuron002
FatJet_inclParTMDV1_hidNeuron003
FatJet_inclParTMDV1_hidNeuron004
FatJet_inclParTMDV1_hidNeuron005
FatJet_inclParTMDV1_hidNeuron006
FatJet_inclParTMDV1_hidNeuron007
FatJet_inclParTMDV1_hidNeuron008
FatJet_inclParTMDV1_hidNeuron009
FatJet_inclParTMDV1_hidNeuron010
FatJet_inclParTMDV1_hidNeuron011
FatJet_inclParTMDV1_hidNeuron012
FatJet_inclParTMDV1_hidNeuron013
FatJet_inclParTMDV1_hidNeuron014
FatJet_inclParTMDV1_hidNeuron015
FatJet_inclParTMDV1_hidNeuron016
FatJet_inclParTMDV1_hidNeuron017
FatJet_inclParTMDV1_hidNeuron018
FatJet_inclParTMDV1_hidNeuron019
FatJet_inclParTMDV1_hidNeuron020
FatJet_inclParTMDV1_hidNeuron021
FatJet_inclParTMDV1_hidNeuron022
FatJet_inclParTMDV1_hidNeuron023
FatJet_inclPar

In [23]:
! ls ../outfiles/0-1_mu.parquet

0-1_mu.parquet	0-1.pkl


In [28]:
# keys in parquet
for key in pd.read_parquet("../outfiles/0-1_mu.parquet").keys():
    print(key)

met
ht
nfj
nj
deta
mjj
fj_genH_pt
fj_genV_dR
fj_genVstar
genV_genVstar_dR
fj_nprongs
fj_ncquarks
fj_lepinprongs
fj_H_VV_4q
fj_H_VV_elenuqq
fj_H_VV_munuqq
fj_H_VV_leptauelvqq
fj_H_VV_leptaumuvqq
fj_H_VV_hadtauvqq
fj_H_VV_isVlepton
fj_H_VV_isVstarlepton
fj_H_VV_isMatched
weight
weight_genweight
weight_L1Prefiring
weight_pileup
mu_mvaId
mu_highPtId
weight_trigger_iso_muon
weight_trigger_noniso_muon
weight_isolation_muon
weight_id_muon
weight_mu
fj_pt
fj_msoftdrop
fj_bjets_ophem
fj_bjets
lep_pt
lep_isolation
lep_misolation
lep_fj_m
lep_fj_dr
lep_met_mt
met_fj_dphi
rec_higgs_m


In [4]:
def pad_val(
    arr: ak.Array,
    value: float,
    target: int = None,
    axis: int = 0,
    to_numpy: bool = False,
    clip: bool = True,
):
    """
    basically: preserves the nested structure of the ak array and replaces None values with -1
    pads awkward array up to ``target`` index along axis ``axis`` with value ``value``,
    optionally converts to numpy array
    """
    if target:
        ret = ak.fill_none(ak.pad_none(arr, target, axis=axis, clip=clip), value, axis=None)
    else:
        ret = ak.fill_none(arr, value, axis=None)
    return ret.to_numpy() if to_numpy else ret

In [5]:
def build_p4(cand):
    return ak.zip(
        {
            "pt": cand.pt,
            "eta": cand.eta,
            "phi": cand.phi,
            "mass": cand.mass,
            "charge": cand.charge,
        },
        with_name="PtEtaPhiMCandidate",
        behavior=candidate.behavior,
    )

In [6]:
nevents = len(events)

# define muon objects
loose_muons = (
    (((events.Muon.pt > 30) & (events.Muon.pfRelIso04_all < 0.25)) |
     (events.Muon.pt > 55))
    & (np.abs(events.Muon.eta) < 2.4)
    & (events.Muon.looseId)
)
n_loose_muons = ak.sum(loose_muons, axis=1)

good_muons = (
    (events.Muon.pt > 28)
    & (np.abs(events.Muon.eta) < 2.4)
    & (np.abs(events.Muon.dz) < 0.1)
    & (np.abs(events.Muon.dxy) < 0.05)
    & (events.Muon.sip3d <= 4.0)
    & events.Muon.mediumId
)
n_good_muons = ak.sum(good_muons, axis=1)

# define electron objects
loose_electrons = (
    (((events.Electron.pt > 38) & (events.Electron.pfRelIso03_all < 0.25)) |
     (events.Electron.pt > 120))
    & ((np.abs(events.Electron.eta) < 1.44) | (np.abs(events.Electron.eta) > 1.57))
    & (events.Electron.cutBased >= events.Electron.LOOSE)
)
n_loose_electrons = ak.sum(loose_electrons, axis=1)

good_electrons = (
    (events.Electron.pt > 38)
    & ((np.abs(events.Electron.eta) < 1.44) | (np.abs(events.Electron.eta) > 1.57))
    & (np.abs(events.Electron.dz) < 0.1)
    & (np.abs(events.Electron.dxy) < 0.05)
    & (events.Electron.sip3d <= 4.0)
    & (events.Electron.mvaFall17V2noIso_WP90)
)
n_good_electrons = ak.sum(good_electrons, axis=1)

# leading lepton
goodleptons = ak.concatenate([events.Muon[good_muons], events.Electron[good_electrons]], axis=1)
goodleptons = goodleptons[ak.argsort(goodleptons.pt, ascending=False)]
candidatelep = ak.firsts(goodleptons)

In [7]:
# candidate leptons
candidatelep_p4 = build_p4(candidatelep)

# JETS
goodjets = events.Jet[
    (events.Jet.pt > 30)
    & (abs(events.Jet.eta) < 2.5)
    & events.Jet.isTight
    & (events.Jet.puId > 0)
]
ht = ak.sum(goodjets.pt, axis=1)

# FATJETS
fatjets = events.FatJet

good_fatjets = (
    (fatjets.pt > 200)
    & (abs(fatjets.eta) < 2.5)
    & fatjets.isTight
)
n_fatjets = ak.sum(good_fatjets, axis=1)

good_fatjets = fatjets[good_fatjets]
good_fatjets = good_fatjets[ak.argsort(good_fatjets.pt, ascending=False)]

# for leptonic channel: leading pt which contains lepton
no_lep_in_fj_bool = good_fatjets.delta_r(candidatelep_p4)>0.8
good_fatjets = good_fatjets[no_lep_in_fj_bool]
candidatefj_lep = ak.firsts(good_fatjets[ak.argmin(good_fatjets.delta_r(candidatelep_p4), axis=1, keepdims=True)])

In [8]:
candidatefj_lep.pt

<Array [286, 314, 310, 251, ... None, 347, 251] type='31766 * ?float32[parameter...'>

In [9]:
candidatefj_lep.pt

<Array [286, 314, 310, 251, ... None, 347, 251] type='31766 * ?float32[parameter...'>

In [10]:
met = events.MET
mt_lep_met = np.sqrt(
    2. * candidatelep_p4.pt * met.pt * (ak.ones_like(met.pt) - np.cos(candidatelep_p4.delta_phi(met)))
)

In [19]:
good_fatjets.delta_phi(met)

<Array [[1.88], [-0.628], ... [-2.85, 1.59]] type='31766 * option[var * float32]'>

In [15]:
good_fatjets

<FatJetArray [[FatJet], ... [FatJet, FatJet]] type='31766 * option[var * fatJet]'>